In [10]:
# from moccasin import setup_notebook

# setup_notebook()

In [11]:
from moccasin.config import get_active_network

active_network = get_active_network()
print(active_network.name)

mainnet-fork


In [12]:
import boa
from moccasin.config import get_config
from boa.contracts.abi.abi_contract import ABIContract
from typing import Tuple

STARTING_ETH_BALANCE = int(1000e18)
STARTING_USDC_BALANCE = int(100e6)
STARTING_WETH_BALANCE = int(1e18)

def _add_eth_balance():
    boa.env.set_balance(boa.env.eoa, STARTING_ETH_BALANCE)

def _add_token_balance(usdc, weth, active_network):
   
    weth.deposit(value=STARTING_WETH_BALANCE)
    our_address = boa.env.eoa
    
    with boa.env.prank(usdc.owner()):
        usdc.updateMasterMinter(our_address)
        
    usdc.configureMinter(our_address, STARTING_USDC_BALANCE)
    usdc.mint(our_address, STARTING_USDC_BALANCE)
    
    print(usdc.balanceOf(our_address))
    
    
def setup_script() -> Tuple[ABIContract,ABIContract,ABIContract,ABIContract]:
    print("Starting setup script")
    
    active_network = get_config().get_active_network()
    
    usdc = active_network.manifest_named("usdc")
    weth = active_network.manifest_named("weth")

    if active_network.is_local_or_forked_network():
        _add_eth_balance()
        _add_token_balance(usdc, weth, active_network)

    print(usdc.balanceOf(boa.env.eoa))
 
def moccasin_main():
    setup_script()

moccasin_main()

Starting setup script
300000000
300000000


In [13]:
usdc = active_network.manifest_named("usdc")
weth = active_network.manifest_named("weth")

In [14]:
usdc.balanceOf(boa.env.eoa)
weth.balanceOf(boa.env.eoa)

3000000000000000000